In [1]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
from speckle_filter import leesigma

ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VMpmBKt5kR2-MsX3LChbrvBCoV9eT81tUTgn1yazio4GcK7Wolmmuc

Successfully saved authorization token.


In [67]:
# define region and dates of interest
# roi = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  # Nanda Bet small region
# roi2 = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271])  # Nanda Bet S1 extent
roi = ee.Geometry.Rectangle([70.3145, 23.0592, 72.4516, 24.2214])  # Nanda Bet - CYGNSS & S1 common extent for fusion

d1 = '2020-12-04'
d2 = '2020-12-05'

# d1 = '2020-08-18'
# d2 = '2020-08-19'

# get s1 data
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filterMetadata('resolution_meters', 'equals', 10)
                 .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                 .filter(ee.Filter.date(d1,d2))
                 .filterBounds(roi))

n = s1_collection.size().getInfo()
print('Number of images:', n)
print('Bands:', s1_collection.first().bandNames().getInfo())
                

Number of images: 1
Bands: ['VV', 'VH', 'angle']


In [68]:
# get dates of images
acq_times = s1_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
date_list

['12-04-2020']

In [69]:
# implement speckle filter and get vv/vh ratio
def speckle(img):
    from speckle_filter import leesigma
    k = 5
    vv = img.select('VV')
    vh = img.select('VH')
    vv_vh = vv.divide(vh).rename('VV_VH')
    vv_filtered = leesigma(vv,k).rename('VV_filtered')
    vh_filtered = leesigma(vh,k).rename('VH_filtered')
    vv_vh_filtered = vv_filtered.divide(vh_filtered).rename('VV_VH_filtered')
    img = img.addBands(ee.Image([vv_vh, vv_filtered, vh_filtered, vv_vh_filtered]))
    return img

s1_collection = s1_collection.map(speckle)
#print(s1_collection.first().bandNames().getInfo())

In [61]:
# map to make sure it looks right

# define visualization parameters
vv_vis = {
    'min': 0,
    'max': 0.2,
    'palette': ['#000000','#ffffff']
}

vh_vis = {
    'min': 0,
    'max': 0.03,
    'palette': ['#000000','#ffffff']
}

vv_vh_vis = {
    'min': 3,
    'max': 30,
    'palette': ['#000000','#ffffff']
}



# Select individual images to display on map

s1_collection_list = s1_collection.toList(n)

idx = 0  #get first image in collection
vv = ee.Image(s1_collection_list.get(idx)).select('VV_filtered')
vh = ee.Image(s1_collection_list.get(idx)).select('VH_filtered')
vv_vh = ee.Image(s1_collection_list.get(idx)).select('VV_VH_filtered')
vv_unfiltered = ee.Image(s1_collection_list.get(idx)).select('VV')


# Create map and add layers
M = geemap.Map()
M.centerObject(roi, 8)

M.addLayer(vv, vv_vis, 'VV')
M.addLayer(vh, vh_vis, 'VH')
M.addLayer(vv_vh, vv_vh_vis, 'VV/VH')
M.addLayer(vv_unfiltered, vv_vis, 'VV Unfiltered')

M  # show map


Map(center=[23.643099947449006, 71.38305000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [50]:
ns = ee.Image(s1_collection_list.get(0)).select('VV_filtered').projection().nominalScale().getInfo()
print(ns)

10


In [70]:
# export images
s1_collection_list = s1_collection.toList(n)

i = 0
#print(ee.Image(s1_collection_list.get(i)).select('VV_filtered').projection().crs().getInfo())
ns = ee.Image(s1_collection_list.get(i)).select('VV_filtered').projection().nominalScale().getInfo()
#print(ns)
task_id = []

for i in range(n):
    img = ee.Image(s1_collection_list.get(i)).select(['VV_filtered','VH_filtered','VV_VH_filtered']).clip(roi)
    txt = 's1_nanda_bet_' + date_list[i] + '_100m'
    task = ee.batch.Export.image.toDrive(image=img,
                                        description=txt,
                                        scale=100,
                                        region=roi,
                                        fileNamePrefix=txt,
                                        crs='EPSG:4326',
                                        fileFormat='GeoTIFF',
                                        maxPixels=1e9)
    task.start()
    task_id.append(task.id)
    print(date_list[i])
    


12-04-2020


In [34]:
#task.status()
t_id = "B7GV2MKIIIZ7EPVO5632SPKU"
print(ee.data.getTaskStatus(t_id))

t = ee.data.getTaskStatus("B7GV2MKIIIZ7EPVO5632SPKU")[0].get('creation_timestamp_ms')
print('\nTask started on:', time.strftime('%m-%d-%Y', time.gmtime(t/1000)))

[{'state': 'COMPLETED', 'description': 's1_nanda_bet_01-03-2020', 'creation_timestamp_ms': 1663260176170, 'update_timestamp_ms': 1663261123605, 'start_timestamp_ms': 1663260222878, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'batch_eecu_usage_seconds': 6100.69580078125, 'id': 'B7GV2MKIIIZ7EPVO5632SPKU', 'name': 'projects/earthengine-legacy/operations/B7GV2MKIIIZ7EPVO5632SPKU'}]

Task started on: 09-15-2022


In [52]:
task.list()

[<Task VLCF6GE7ZW5YQYWKLPOUXTMH EXPORT_IMAGE: s1_nanda_bet_07-13-2020_100m (RUNNING)>,
 <Task Q2IJ2RYRLVAXV2GJJOWYVRGU EXPORT_IMAGE: s1_nanda_bet_02-08-2020_100m (COMPLETED)>,
 <Task L5LUQLCVHR6METJHVIDCB6AQ EXPORT_IMAGE: s1_nanda_bet_02-20-2020_100m (COMPLETED)>,
 <Task OS2CILXXFQVFZDX6TR6B23DJ EXPORT_IMAGE: s1_nanda_bet_03-03-2020_100m (COMPLETED)>,
 <Task UJTNUM3TYS5ID7MWBJO4DEAJ EXPORT_IMAGE: s1_nanda_bet_01-27-2020_100m (COMPLETED)>,
 <Task 6F3IG2LKZAULZXTLAGZWZBCH EXPORT_IMAGE: s1_nanda_bet_01-15-2020_100m (COMPLETED)>,
 <Task OI4LKKI2SP4DZHD5NBBQS3WM EXPORT_IMAGE: s1_nanda_bet_07-25-2020 (COMPLETED)>,
 <Task 4I3VDJZJNNCL3SIEMCXSPI7D EXPORT_IMAGE: s1_nanda_bet_08-06-2020 (COMPLETED)>,
 <Task 2T5ZWQ6EP6SG7DGAFMFMEPPW EXPORT_IMAGE: s1_nanda_bet_08-18-2020 (COMPLETED)>,
 <Task ORK4YRQNNZFNKJGG7ZKDOHET EXPORT_IMAGE: copernicus_land_cover_nanda_bet_2019 (COMPLETED)>,
 <Task AVYCYKJA7L4V7RQRL4NHHC7G EXPORT_IMAGE: copernicus_land_cover_nanda_bet_2019 (COMPLETED)>,
 <Task LTU3NR6YAHYJISP

In [77]:
(ee.data.getTaskStatus(task_id[1])[0]).get('state')

'READY'